In [1]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout, GlobalAveragePooling2D, MaxPooling2D, Concatenate, Flatten, Dense
from tensorflow.keras.applications import vgg16

from tensorflow.keras.applications import resnet

from tensorflow.keras import applications

from tensorflow.keras import layers,losses,optimizers,metrics,Model

import tensorflow as tf

from tensorflow.keras.utils import plot_model

import tensorflow as tf
from tensorflow.keras.initializers import glorot_uniform
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
import os


In [2]:
import numpy as np
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
def create_batch_data(train_resampled,label,batch_size):
    
        
    def data_augmentation(data,image_count=1):

        augmented_images=[]
        
        patch_size=512

        datagen=ImageDataGenerator(horizontal_flip=True)

        it=datagen.flow(np.expand_dims(data,axis=0),batch_size=1)

        for i in range(image_count):
    
            augmented_images.append(np.squeeze(it.next(),axis=0))
        
        return(np.stack(augmented_images)) 
    
    anchors,refs,events=np.zeros((batch_size,patch_size,patch_size,3)),np.zeros((batch_size,patch_size,patch_size,3)),np.zeros((batch_size,patch_size,patch_size,3))
    
    for i in range(batch_size):
 
            index=random.randint(0,train_resampled.shape[0]-1)

            #anchor=data_augmentation(positive)

            ref=train_resampled[index,:,:,0:3]
            
            event=train_resampled[index,:,:,3:]

            anchor=data_augmentation(ref)

            anchors[i],refs[i],events[i]=anchor,ref,event
    
    return [anchors,refs,events]


In [ ]:
def Contrastive_Loss(margin,embedding_dim,tav): #tav:maximum acceptable distance between similar samples
    
    def get_loss(output_true,output_pred):
        
        output_true=tf.cast(output_true,tf.float32)
        
        input1=output_pred[:,:embedding_dim]
        
        input2=output_pred[:,embedding_dim:2*embedding_dim]
        
        d=tf.reduce_sum(tf.square(input1-input2))
        
        return(output_true*tf.maximum(margin-d,0)+(1-output_true)*tf.maximum(d-tav,0))  
    
    return get_loss

In [4]:
def Data_Generator(train_resampled,label,batch_size,embedding_dim,mode):
    
    while True:
    
        x=create_batch_data(train_resampled,label,batch_size)
        
        if mode=='triplet':
        
            y=np.zeros((batch_size,3*embedding_dim))
            
        else:
            
            x=x[1:]
            
            y=np.zeros((batch_size,2*embedding_dim))
        
        yield x,y

In [5]:
tf.compat.v1.enable_eager_execution()

patch_size=512

#cnn=vgg16.VGG16(weights='imagenet',input_shape=(patch_size,patch_size)+(3,),include_top=False)

flatten=layers.Flatten()(cnn.output)

#tc1 = layers.Conv2DTranspose(256, 17, activation='relu', data_format='channels_last')(cnn.output)

#tc1=layers.BatchNormalization()(tc1)

dense_1=layers.Dense(512,activation='relu')(flatten)

dense_1=layers.BatchNormalization()(dense_1)

dense_2=layers.Dense(256,activation='relu')(dense_1)

#tc2=layers.Conv2DTranspose(64, 97, activation='relu',data_format='channels_last')(tc1)

#tc2=layers.BatchNormalization()(tc2)

dense_2=layers.BatchNormalization()(dense_2)

#output=layers.Conv2DTranspose(3,129, activation='relu', data_format='channels_last')(tc2)

output=layers.Dense(256)(dense_2)



embedding=Model(cnn.input,output,name='Embedding')

embedding.summary()

Model: "Embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0 

In [10]:
# ### Creating Siamese Network Architecture With Triplets Of Anchor, Positive, and Negative Image Patches as Input

anchor_input=layers.Input(name='anchor',shape=(patch_size,patch_size)+(3,))

positive_input=layers.Input(name='positive',shape=(patch_size,patch_size)+(3,))

negative_input=layers.Input(name='negative',shape=(patch_size,patch_size)+(3,))

embedding_anchor=embedding(anchor_input)
embedding_positive=embedding(positive_input)
embedding_negative=embedding(negative_input)

output=layers.concatenate([embedding_anchor,embedding_positive,embedding_negative])
#output=layers.concatenate([embedding_positive,embedding_negative])

model=Model([anchor_input,positive_input,negative_input],output)
#model=Model([positive_input,negative_input],output)

In [7]:
### Triplet Loss

def Triplet_Loss(margin, embedding_dim):
    
    def get_loss(output_true,output_pred):
        
        anchor_output=output_pred[:,:embedding_dim]
        
        positive_output=output_pred[:,embedding_dim:2*embedding_dim]
        
        negative_output=output_pred[:,2*embedding_dim:]
        
        dp=tf.reduce_sum(tf.square(anchor_output-positive_output),axis=1)
        
        dn=tf.reduce_sum(tf.square(anchor_output-negative_output),axis=1)
        
        return tf.maximum(dp-dn+margin,0)
        
    return get_loss    


### Contrastive Loss

def Contrastive_Loss(margin,embedding_dim,tav): #tav:maximum acceptable distance between similar samples
    
    def get_loss(output_true,output_pred):

        
        output_true=tf.cast(output_true,tf.float32)
        
        input1=output_pred[:,:embedding_dim]
        
        input2=output_pred[:,embedding_dim:2*embedding_dim]
        
        d=tf.reduce_sum(tf.square(input1-input2))
        
        return(output_true*tf.maximum(margin-d,0)+(1-output_true)*tf.maximum(d-tav,0))  
    
    return get_loss


### Weighted Double Margin Contrastive Loss (WDMCL)

def Weighted_Double_Margin_Contrastive_Loss(embedding_dim,w1,w2 ,m1=0.9,m2=0.45):
    
    def get_loss(output_true,output_pred):

        
        input1=output_pred[:,:embedding_dim]
    
        input2=output_pred[:,embedding_dim:2*embedding_dim]
        
        d=tf.reduce_sum(tf.square(input1-input2))
        
        return(0.5*((w1*(1-output_true)*(tf.maximum(d-m1,0))**2)+(w2*output_true*(tf.maximum(m2-d,0))**2)))
    
    return(get_loss)       

In [ ]:
def Data_Generator(train_resampled,label,batch_size,embedding_dim,mode):
    
    while True:
    
        x=create_batch_data(train_resampled,label,batch_size)
        
        if mode=='triplet':
        
            y=np.zeros((batch_size,3*embedding_dim))
            
        else:
            
            x=x[1:]
            
            y=np.zeros((batch_size,2*embedding_dim))
        
        yield x,y

In [12]:
# # Fitting The Siamese Model and Setting Training Parameters

#train_resampled__s,_,train_label_s,_=train_test_split(train_resampled_,train_label,stratify=train_label,test_size=0.96,random_state=1)

#val_resampled__s,_,val_label_s,_=train_test_split(val_resampled,val_label,stratify=val_label,test_size=0.96,random_state=1)

batch_size,embedding_dim, epochs=5, 256, 200

lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(0.001,decay_rate=0.8,decay_steps=10000)

#model.compile(loss=Triplet_Loss(margin=0.9, embedding_dim=embedding_dim),optimizer=Adam(learning_rate=lr_schedule))

        
#w1=1/(len(train_label_s[train_label_s==0])) # weight for unchanged pixels
        
#w2=1/(len(train_label_s[train_label_s==1])) # weight for changed pixels

#w1=1/(len(train_label[train_label==0])) # weight for unchanged pixels
        
#w2=1/(len(train_label[train_label==1])) # weight for changed pixels


#Weighted_Double_Margin_Contrastive_Loss(embedding_dim,w1,w2,m1=0.9,m2=0.45)

#Contrastive_Loss(embedding_dim=embedding_dim,margin=0.01,tav=0.01)

model.compile(loss=Triplet_Loss(margin=0.9, embedding_dim=embedding_dim),optimizer=Adam(learning_rate=lr_schedule))

es=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,min_delta=1e-3,restore_best_weights=True)

In [ ]:
import pickle

for i in range(1,6):

    imgs = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_image/fold_{i}/train/fold_{i}.npy')
    
    masks = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_mask/fold_{i}/train/fold_{i}.npy')

    imgs_valid = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_image/fold_{i}/valid/fold_{i}.npy')
    
    masks_valid = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_mask/fold_{i}/valid/fold_{i}.npy')
    
    mch=tf.keras.callbacks.ModelCheckpoint(filepath=f'G:/sen1flood11_triplet_loss_fold_{i}.h5',save_best_only=True)

    
    data_valid=create_batch_data(imgs_valid,masks_valid, imgs_valid.shape[0])
    
    #data_valid = data_valid[1:] #omitting the anchor 
    
    history=model.fit(Data_Generator(imgs,masks,batch_size,embedding_dim,'triplet'),steps_per_epoch=int(imgs.shape[0]/batch_size),epochs=epochs,callbacks=[es,mch],validation_data=(data_valid,masks_valid))
    
    ##history=model.fit(Data_Generator(train_resampled,train_label,batch_size,'contrastive'),steps_per_epoch=int(train_resampled.shape[0]/batch_size),epochs=epochs,callbacks=[es,mch],validation_data=(val_data,val_label))
    
    
    model.save(f'G:/sen1flood11_triplet_loss_fold_{i}_last_epoch.h5')


    with open(f'G:/sen1flood11_triplet_loss_fold_{i}.pckl', 'wb') as hist:
        
        pickle.dump(history.history,hist)

Epoch 1/200
62/62 [==============================] - ETA: 0s - loss: 3.7129 

In [ ]:
from sklearn.utils import shuffle

from tensorflow.keras.layers import LeakyReLU, Conv2DTranspose

from tensorflow.keras import layers,Model

embedding_dim=256

patch_size=512

input_layer=layers.Input(shape=(2*embedding_dim,))

#flatten1=layers.Flatten()(input_layer)

#dense1=layers.Dense(256)(flatten1)

input_layer_r=tensorflow.reshape(input_layer, [-1,16,16,2])

tc1 = layers.Conv2DTranspose(256, 17, data_format='channels_last')(input_layer_r)

LR1=LeakyReLU(alpha=0.1)(tc1)

#dense2=layers.Dense(128)(LR1)

tc2 = layers.Conv2DTranspose(128, 33, data_format='channels_last')(LR1)

LR2=LeakyReLU(alpha=0.1)(tc2)

drop1=layers.Dropout(0.1)(LR2)

#dense3=layers.Dense(128)(drop1)

tc3 = layers.Conv2DTranspose(64, 65, data_format='channels_last')(drop1)

LR3=LeakyReLU(alpha=0.1)(tc3)

drop2=layers.Dropout(0.1)(LR3)

#dense4=layers.Dense(64)(drop2)

tc4 = layers.Conv2DTranspose(32, 129, data_format='channels_last')(drop2)

LR4=LeakyReLU(alpha=0.1)(tc4)

#dense5=layers.Dense(32)(LR4)

tc5 = layers.Conv2DTranspose(1, 257, data_format='channels_last')(LR4)

LR5=LeakyReLU(alpha=0.1)(tc5)

#prediction_layer=layers.Dense(2,activation='softmax')(LR5)

prediction_layer = layers.Conv2DTranspose(1,1, activation='softmax')(LR5)

model1=Model(input_layer,prediction_layer,name='prediction_model')


model1.summary()

In [ ]:
train_data=create_batch_data(imgs,masks,imgs.shape[0])

##train_label_pos,train_label_neg=train_label[np.where(train_label==0)[0]],train_label[np.where(train_label==1)[0]]

train_prediction=model.predict(train_data)

val_prediction=model.predict(val_data)

# One Hot Labeling

In [ ]:
from tensorflow.keras.utils import to_categorical

import numpy as np

for i in range(1,2):

    imgs = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_image/fold_{i}/train/fold_{i}.npy')

    masks = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_mask/fold_{i}/train/fold_{i}.npy')

    imgs_valid = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_image/fold_{i}/valid/fold_{i}.npy')

    masks_valid = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_mask/fold_{i}/valid/fold_{i}.npy')

    masks_one_hot = []

    masks_one_hot_valid = []

    for m in range(masks.shape[0]):

        masks_1d = to_categorical(np.ravel(masks[m]),num_classes=2)

        masks_one_hot.append(np.concatenate((np.expand_dims(np.reshape(masks_1d[:,0],(512,512)),axis=2),np.expand_dims(np.reshape(masks_1d[:,1],(512,512)),axis=2)), axis=2))

    masks_one_hot = np.stack(masks_one_hot)

    for mm in range(masks_valid.shape[0]):

        masks_1d_valid = to_categorical(np.ravel(masks_valid[mm]), num_classes=2)

        masks_one_hot_valid.append(np.concatenate((np.expand_dims(np.reshape(masks_1d_valid[:,0],(512,512)),axis=2),np.expand_dims(np.reshape(masks_1d_valid[:,1],(512,512)),axis=2)), axis=2))


    masks_one_hot_valid = np.stack(masks_one_hot_valid)   

    print(masks_one_hot.shape, masks_one_hot_valid.shape)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers import Adam,SGD,Adadelta

from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow.keras.optimizers.schedules import ExponentialDecay

#anchor_embedding_train,positive_embedding_train,negative_embedding_train=train_prediction[:,:256],train_prediction[:,256:2*256],train_prediction[:,2*256:3*256]

#embedding_t,label_t=shuffle(np.concatenate((positive_embedding_train,negative_embedding_train),axis=0),np.concatenate((train_label_pos,train_label_neg),axis=0),random_state=2)

lr_Schedule1=ExponentialDecay(0.001,decay_rate=0.8,decay_steps=1000)

es1=EarlyStopping(monitor='val_loss',min_delta=1e-4,patience=100,restore_best_weights=True)

model1.compile(loss='binary_crossentropy',optimizer=Adam(learning_rate=lr_Schedule1),metrics=['accuracy'])


history1=model1.fit(train_prediction,masks_one_hot,batch_size=80, epochs=300,callbacks=[es1],validation_data=(val_prediction,masks_one_hot_valid))

In [ ]:
def create_change_map(model,model1,imgs_test,embedding_dim=256):

    flood_maps=[]

    for i in range(imgs_test.shape[0]):
        
        embedding_input=create_batch_data(np.expand_dims(imgs_test[i],axis=0))
            
        embedding=model.predict(test_prediction)

        flood_maps.append(np.argmax(model1.predict(embedding),axis=2))    
            
                            
    return(flood_map) 


embedding_dim=256

NUM_FOLD =10

flood_maps = []

for i in range(1,Num_Fold+1):

    imgs_test = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_image/fold_{i}/test/fold_{i}.npy')
    
    masks_test = np.load(f'G:/sen11_cross_validation_bitemporal_numpy_mask/fold_{i}/test/fold_{i}.npy')


    flood_maps.append(create_change_map(model,model1,imgs_test))